In [1]:
import pandas as pd

In [13]:
df = pd.read_csv('data/1016version.csv')

In [190]:
positions = ['SUPPORT', 'ADC', 'MIDDLE', 'JUNGLE', 'TOP']
t1_fields = []

for position in range(1, 6):
    field = f't1p{position}_champId'
    t1_fields.append(field)

t1 = df[t1_fields].copy()
t1['t1_win'] = df['t1_win']
t1 = t1.melt(id_vars=['t1_win'], value_vars=t1_fields[:], var_name='position', value_name='champId')
t1['position'] = t1['position'].replace(dict(zip(t1_fields, positions)))
t1 = t1.groupby(['position', 'champId']).agg({'t1_win': ['count', 'sum', 'mean']})
t1.columns = ['appearances', 'wins', 'winrate']
t1 = t1.reset_index()


In [191]:
t2_fields = []

for position in range(1, 6):
    field = f't2p{position}_champId'
    t2_fields.append(field)

t2 = df[t2_fields].copy()
t2['t2_win'] = df['t1_win'].replace({0: 1, 1: 0})
t2 = t2.melt(id_vars=['t2_win'], value_vars=t2_fields[:], var_name='position', value_name='champId')
t2['position'] = t2['position'].replace(dict(zip(t2_fields, positions)))
t2 = t2.groupby(['position', 'champId']).agg({'t2_win': ['count', 'sum', 'mean']})
t2.columns = ['appearances', 'wins', 'winrate']
t2 = t2.reset_index()

In [192]:
t2

,position,champId,appearances,wins,winrate
0,ADC,1,8,7,0.875000
1,ADC,2,14,9,0.642857
2,ADC,3,12,8,0.666667
3,ADC,7,1,0,0.000000
4,ADC,8,15,6,0.400000
...,...,...,...,...,...
590,TOP,518,288,142,0.493056
591,TOP,555,37,23,0.621622
592,TOP,777,659,319,0.484067
593,TOP,875,947,458,0.483633


In [193]:
t1

,position,champId,appearances,wins,winrate
0,ADC,1,18,8,0.444444
1,ADC,2,13,5,0.384615
2,ADC,3,7,2,0.285714
3,ADC,4,2,1,0.500000
4,ADC,7,2,1,0.500000
...,...,...,...,...,...
597,TOP,523,2,1,0.500000
598,TOP,555,41,24,0.585366
599,TOP,777,675,333,0.493333
600,TOP,875,990,492,0.496970


In [194]:
merged_table = pd.merge(t1, t2, on=['position', 'champId'], how='outer')
merged_table = merged_table.fillna(0)
merged_table['appearances'] = merged_table['appearances_x'].add(merged_table['appearances_y'])
merged_table['wins'] = merged_table['wins_x'].add(merged_table['wins_y'])
merged_table = merged_table.drop(columns=['appearances_x', 'appearances_y', 'wins_x', 'wins_y', 'winrate_x', 'winrate_y'])

In [195]:
merged_table['winrate'] = merged_table['wins'] / merged_table['appearances']

In [196]:
pivot_table = merged_table.pivot_table(index='champId', 
                                        columns='position', 
                                        values=['appearances', 'wins', 'winrate'],
                                        aggfunc='sum')

In [197]:
pivot_table.columns = ['_'.join(reversed(col)).strip() for col in pivot_table.columns.values]
pivot_table = pivot_table.sort_index(axis=1)

In [198]:
pivot_table = pivot_table.reset_index()

In [200]:
pivot_table.to_csv('data/table3.csv', index=False)

In [10]:
names = pd.read_csv('data/names.csv')

In [14]:
for t in range(1, 3):
    for position in range(1, 6):
        field = f't{t}p{position}_champId'
        df[field] = df[field].map(names.set_index('champId')['champName'])
        df = df.rename(columns={field: f't{t}p{position}_champName'})

In [16]:
df.to_csv('data/1016version.csv', index=False)